In [11]:
# Health Dept woodoak ln, Murray
# depot,40.6533382,-111.8720126

# In'n'Out Jordan Landing
# location1,40.6140938,-111.976211

# Taco Time 9000 S 2700 W
# location2,40.5835943,-111.9673276

# Red Maple Chinese 4700 S 2700 W
# location3,40.6731633,-111.9641089

# Topgolf Midvale
# location4,40.6244879,-111.922564

# Lagoon
# 40.9856936,-111.8970475

import pandas as pd
from scipy.spatial import distance_matrix

data = [[40.6533382,-111.8720126], [40.6731633,-111.9641089], [40.6140938,-111.976211], [40.5835943,-111.9673276],
        [40.6244879,-111.922564], [40.9856936,-111.8970475], [40.1690666,-111.679458]]
ctys = ['Health Department', 'Red Maple Chinese', "In 'N Out", 'Taco Time', 'Topgolf', 'Lagoon', 'Flying J']
df = pd.DataFrame(data, columns=['xcord', 'ycord'], index=ctys)

In [12]:
df

,xcord,ycord
Health Department,40.653338,-111.872013
Red Maple Chinese,40.673163,-111.964109
In 'N Out,40.614094,-111.976211
Taco Time,40.583594,-111.967328
Topgolf,40.624488,-111.922564
Lagoon,40.985694,-111.897047
Flying J,40.169067,-111.679458


In [13]:
distance_matrix =  pd.DataFrame(distance_matrix(df.values, df.values), index=df.index, columns=df.index)
distance_matrix

,Health Department,Red Maple Chinese,In 'N Out,Taco Time,Topgolf,Lagoon,Flying J
Health Department,0.000000,0.094206,0.111344,0.118107,0.058205,0.333297,0.521149
Red Maple Chinese,0.094206,0.000000,0.060296,0.089627,0.063994,0.319644,0.578912
In 'N Out,0.111344,0.060296,0.000000,0.031767,0.054645,0.379939,0.534894
Taco Time,0.118107,0.089627,0.031767,0.000000,0.060631,0.408195,0.504680
Topgolf,0.058205,0.063994,0.054645,0.060631,0.000000,0.362106,0.516245
Lagoon,0.333297,0.319644,0.379939,0.408195,0.362106,0.000000,0.845118
Flying J,0.521149,0.578912,0.534894,0.504680,0.516245,0.845118,0.000000


In [14]:
distance_matrix = distance_matrix*10_000
distance_matrix = distance_matrix.values.tolist()

In [144]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 1  # Only one vehicle for this problem
    data['depot'] = 0  # The depot is index 0 in the distance matrix
    return data

In [145]:
# pip install ortools

In [146]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In [147]:
data = create_data_model()
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [148]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [149]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [150]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [151]:
# This the search parameter that takes some extra time to explore outside of any local minima it may encounter

# search_parameters = pywrapcp.DefaultRoutingSearchParameters()
# search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
# search_parameters.time_limit.seconds = 30
# search_parameters.log_search = True

In [152]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

In [153]:
# Print solution for the optimal route

solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Objective: 18237 miles
Route for vehicle 0:
 0 -> 5 -> 1 -> 2 -> 3 -> 6 -> 4 -> 0



In [ ]:
# Next Steps:

# Consolidate code it into a nice function so it can be run simply

# After that, I need to connect it to a data source so I can pull in data and assign dummy inspection dates
# I can use the FourSquare API to pull in several dozen locations at once
# Dates can be assigned randomly in groups of 5, and then the optimizer can pick the soonest 5
# Replicating the date 5 times could be hard, and it's totally contrived anyway. As a workaround, just assign a number between 1 and len(list)
# and take the first 5 items.

# Also, need to check and make sure it's not getting stuck in a local minimum
# Might also be nice to use Manhattan distance instead of Euclidean since it's pretty much on a grid system
# Later draft could incorporate Google Maps API for actual driving routes